In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
import os, sys

In [2]:
print("hello world")

hello world


In [ ]:
!pip install -U keras-cv-attention-models 
!pip install git+https://github.com/shizuo-kaji/CubicalRipser_3dim
!pip install gudhi

from keras_cv_attention_models import coatnet
import cripser
import gudhi
from gudhi import representations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 502 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 82.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shizuo-kaji/CubicalRipser_3dim to /tmp/pip-req-build-yc6wmdre
  Running command git clone -q https://github.com/shizuo-kaji/CubicalRipser_3dim /tmp/pip-req-build-yc6wmdre
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cripser: filename=cripser-0.0.8-cp37-cp37m-linux_x86_64.whl size=138456 sha256=5e2639865487a536899555e6c331bcc821c32a32a3f29bad7d47c0aae319290d
  Stored in directory: /tmp/pip-ephem-wheel-cache-36l1ha12/wheels/7b/8e/b1/bf858a2c6c9b80f3f55e1840fc7752c8dd982ab48f6b337df7
Successful

In [ ]:
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [3]:
!pip install wandb
!wandb login 300316a5295190e1e55dbc98c9b404940941a598
#apikey = 300316a5295190e1e55dbc98c9b404940941a598

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 27.6 MB/s 
     |████████████████████████████████| 168 kB 70.7 MB/s 
     |████████████████████████████████| 182 kB 61.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 74.3 MB/s 
     |████████████████████████████████| 166 kB 75.4 MB/s 
     |████████████████████████████████| 166 kB 76.1 MB/s 
     |████████████████████████████████| 162 kB 77.7 MB/s 
     |████████████████████████████████| 162 kB 63.3 MB/s 
     |████████████████████████████████| 158 kB 81.1 MB/s 
     |████████████████████████████████| 157 kB 79.3 MB/s 
     |████████████████████████████████| 157 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 82.5 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 157 kB 81.1 MB/s 
     |███████████████████████████

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras


def reload_model_weights(model, pretrained_dict, sub_release, pretrained="imagenet", mismatch_class=None, request_resolution=-1, method="nearest"):
    if pretrained is None:
        return
    if isinstance(pretrained, str) and pretrained.endswith(".h5"):
        print(">>>> Load pretrained from:", pretrained)
        # model.load_weights(pretrained, by_name=True, skip_mismatch=True)
        load_weights_with_mismatch(model, pretrained, mismatch_class, request_resolution, method)
        return pretrained

    file_hash = pretrained_dict.get(model.name, {}).get(pretrained, None)
    if file_hash is None:
        print(">>>> No pretrained available, model will be randomly initialized")
        return None

    if isinstance(file_hash, dict):
        # file_hash is a dict like {224: "aa", 384: "bb", 480: "cc"}
        if request_resolution == -1:
            input_height = model.input_shape[1]
            if input_height is None:  # input_shape is (None, None, 3)
                request_resolution = max(file_hash.keys())
            else:
                request_resolution = min(file_hash.keys(), key=lambda ii: abs(ii - input_height))
        pretrained = "{}_".format(request_resolution) + pretrained
        file_hash = file_hash[request_resolution]
        # print(f"{request_resolution = }, {pretrained = }, {file_hash = }")
    elif request_resolution == -1:
        request_resolution = 224  # Default is 224

    pre_url = "https://github.com/leondgarse/keras_cv_attention_models/releases/download/{}/{}_{}.h5"
    url = pre_url.format(sub_release, model.name, pretrained)
    file_name = os.path.basename(url)
    try:
        pretrained_model = keras.utils.get_file(file_name, url, cache_subdir="models", file_hash=file_hash)
    except:
        print("[Error] will not load weights, url not found or download failed:", url)
        return None
    else:
        print(">>>> Load pretrained from:", pretrained_model)
        # model.load_weights(pretrained_model, by_name=True, skip_mismatch=True)
        load_weights_with_mismatch(model, pretrained_model, mismatch_class, request_resolution, method)
        return pretrained_model


def load_weights_with_mismatch(model, weight_file, mismatch_class=None, request_resolution=-1, method="nearest"):
    model.load_weights(weight_file, by_name=True, skip_mismatch=True)
    input_height, input_width = 244, 244
    # if mismatch_class is not None:
    if mismatch_class is not None and (request_resolution != input_height or request_resolution != input_width):
        try:
            import h5py

            print(">>>> Reload mismatched weights: {} -> {}".format(request_resolution, (input_height, input_width)))
            ff = h5py.File(weight_file, mode="r")
            weights = ff["model_weights"]
            for ii in model.layers:
                if isinstance(ii, mismatch_class) and ii.name in weights:
                    print(">>>> Reload layer:", ii.name)
                    ss = weights[ii.name]
                    # ss = {ww.decode().split("/")[-1] : tf.convert_to_tensor(ss[ww]) for ww in ss.attrs['weight_names']}
                    ss = {ww.decode("utf8") if hasattr(ww, "decode") else ww: tf.convert_to_tensor(ss[ww]) for ww in ss.attrs["weight_names"]}
                    ss = {kk.split("/")[-1]: vv for kk, vv in ss.items()}
                    model.get_layer(ii.name).load_resized_pos_emb(ss, method=method)
            ff.close()

        # print(">>>> Reload mismatched PositionalEmbedding weights: {} -> {}".format(request_resolution, input_shape[0]))
        # bb = keras.models.load_model(pretrained_model)
        # for ii in model.layers:
        #     if isinstance(ii, mismatch_class):
        #         print(">>>> Reload layer:", ii.name)
        #         model.get_layer(ii.name).load_resized_pos_emb(bb.get_layer(ii.name))
        except:
            print("[Error] something went wrong in load_weights_with_mismatch")
            pass



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5
TF_BATCH_NORM_EPSILON = 0.001
LAYER_NORM_EPSILON = 1e-5
CONV_KERNEL_INITIALIZER = keras.initializers.VarianceScaling(scale=2.0, mode="fan_out", distribution="truncated_normal")
# CONV_KERNEL_INITIALIZER = 'glorot_uniform'

from keras_cv_attention_models.attention_layers import (
    activation_by_name,
    batchnorm_with_activation,
    conv2d_no_bias,
    depthwise_conv2d_no_bias,
    drop_block,
    layer_norm,
    se_module,
    output_block,
    MultiHeadRelativePositionalEmbedding,
    add_pre_post_process,
)
# from keras_cv_attention_models.download_and_load import reload_model_weights

PRETRAINED_DICT = {"coatnet0": {"imagenet": {160: "bc4375d2f03b99ac4252770331f0d22f", 224: "29213248739d600cc526c11a79d06775"}}}


def mhsa_with_multi_head_relative_position_embedding(
    inputs, num_heads=4, key_dim=0, global_query=None, out_shape=None, out_weight=True, qkv_bias=False, out_bias=False, attn_dropout=0, name=None
):
    _, hh, ww, cc = inputs.shape
    key_dim = key_dim if key_dim > 0 else cc // num_heads
    qk_scale = float(1.0 / tf.math.sqrt(tf.cast(key_dim, "float32")))
    out_shape = cc if out_shape is None or not out_weight else out_shape
    qk_out = num_heads * key_dim
    # vv_dim = out_shape // num_heads
    vv_dim = key_dim

    if global_query is not None:
        # kv = keras.layers.Dense(qk_out * 2, use_bias=qkv_bias, name=name and name + "kv")(inputs)
        kv = conv2d_no_bias(inputs, qk_out * 2, kernel_size=1, use_bias=qkv_bias, name=name and name + "kv_")
        kv = tf.reshape(kv, [-1, kv.shape[1] * kv.shape[2], kv.shape[-1]])
        key, value = tf.split(kv, [qk_out, out_shape], axis=-1)
        query = global_query
    else:
        # qkv = keras.layers.Dense(qk_out * 3, use_bias=qkv_bias, name=name and name + "qkv")(inputs)
        # qkv = conv2d_no_bias(inputs, qk_out * 2 + out_shape, kernel_size=1, name=name and name + "qkv_")
        qkv = conv2d_no_bias(inputs, qk_out * 3, kernel_size=1, use_bias=qkv_bias, name=name and name + "qkv_")
        qkv = tf.reshape(qkv, [-1, inputs.shape[1] * inputs.shape[2], qkv.shape[-1]])
        query, key, value = tf.split(qkv, [qk_out, qk_out, qk_out], axis=-1)
        # print(f"{query.shape = }, {key.shape = }, {value.shape = }, {num_heads = }, {key_dim = }, {vv_dim = }")
        query = tf.transpose(tf.reshape(query, [-1, query.shape[1], num_heads, key_dim]), [0, 2, 1, 3])  #  [batch, num_heads, hh * ww, key_dim]
    key = tf.transpose(tf.reshape(key, [-1, key.shape[1], num_heads, key_dim]), [0, 2, 3, 1])  # [batch, num_heads, key_dim, hh * ww]
    value = tf.transpose(tf.reshape(value, [-1, value.shape[1], num_heads, vv_dim]), [0, 2, 1, 3])  # [batch, num_heads, hh * ww, vv_dim]

    attention_scores = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([query, key]) * qk_scale  # [batch, num_heads, hh * ww, hh * ww]
    # print(f"{query.shape = }, {key.shape = }, {value.shape = }, {attention_scores.shape = }, {hh = }")
    attention_scores = MultiHeadRelativePositionalEmbedding(with_cls_token=False, attn_height=hh, name=name and name + "pos_emb")(attention_scores)
    attention_scores = keras.layers.Softmax(axis=-1, name=name and name + "attention_scores")(attention_scores)
    attention_scores = keras.layers.Dropout(attn_dropout, name=name and name + "attn_drop")(attention_scores) if attn_dropout > 0 else attention_scores

    # value = [batch, num_heads, hh * ww, vv_dim], attention_output = [batch, num_heads, hh * ww, vv_dim]
    attention_output = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([attention_scores, value])
    attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
    attention_output = tf.reshape(attention_output, [-1, inputs.shape[1], inputs.shape[2], num_heads * vv_dim])
    # print(f">>>> {attention_output.shape = }, {attention_scores.shape = }")

    if out_weight:
        # [batch, hh, ww, num_heads * vv_dim] * [num_heads * vv_dim, out] --> [batch, hh, ww, out]
        attention_output = keras.layers.Dense(out_shape, use_bias=out_bias, name=name and name + "output")(attention_output)
    # attention_output = keras.layers.Dropout(output_dropout, name=name and name + "out_drop")(attention_output) if output_dropout > 0 else attention_output
    return attention_output


def res_MBConv(
    inputs,
    output_channel,
    conv_short_cut=True,
    strides=1,
    expansion=4,
    se_ratio=0,
    drop_rate=0,
    use_dw_strides=True,
    bn_act_first=False,
    activation="gelu",
    name="",
):
    """x ← Proj(Pool(x)) + Conv (DepthConv (Conv (Norm(x), stride = 2))))"""
    preact = batchnorm_with_activation(inputs, activation=None, zero_gamma=False, name=name + "preact_")

    if conv_short_cut:
        shortcut = keras.layers.MaxPool2D(strides, strides=strides, padding="SAME", name=name + "shortcut_pool")(inputs) if strides > 1 else inputs
        shortcut = conv2d_no_bias(shortcut, output_channel, 1, strides=1, name=name + "shortcut_")
        # shortcut = batchnorm_with_activation(shortcut, activation=activation, zero_gamma=False, name=name + "shortcut_")
    else:
        shortcut = inputs

    # MBConv
    input_channel = inputs.shape[-1]
    conv_strides, dw_strides = (1, strides) if use_dw_strides else (strides, 1)  # May swap stirdes with DW
    nn = conv2d_no_bias(preact, input_channel * expansion, 1, strides=conv_strides, use_bias=bn_act_first, padding="same", name=name + "expand_")
    nn = batchnorm_with_activation(nn, activation=activation, act_first=bn_act_first, name=name + "expand_")
    nn = depthwise_conv2d_no_bias(nn, 3, strides=dw_strides, use_bias=bn_act_first, padding="same", name=name + "MB_")
    nn = batchnorm_with_activation(nn, activation=activation, act_first=bn_act_first, zero_gamma=False, name=name + "MB_dw_")
    if se_ratio:
        nn = se_module(nn, se_ratio=se_ratio / expansion, activation=activation, name=name + "se_")
    nn = conv2d_no_bias(nn, output_channel, 1, strides=1, padding="same", name=name + "MB_pw_")
    # nn = batchnorm_with_activation(nn, activation=None, zero_gamma=True, name=name + "MB_pw_")
    nn = drop_block(nn, drop_rate=drop_rate, name=name)
    return keras.layers.Add(name=name + "output")([shortcut, nn])


def res_ffn(inputs, expansion=4, kernel_size=1, drop_rate=0, activation="gelu", name=""):
    """x ← x + Module (Norm(x)), similar with typical MLP block"""
    # preact = batchnorm_with_activation(inputs, activation=None, zero_gamma=False, name=name + "preact_")
    preact = layer_norm(inputs, name=name + "preact_")

    input_channel = inputs.shape[-1]
    nn = conv2d_no_bias(preact, input_channel * expansion, kernel_size, name=name + "1_")
    nn = activation_by_name(nn, activation=activation, name=name)
    nn = conv2d_no_bias(nn, input_channel, kernel_size, name=name + "2_")
    nn = drop_block(nn, drop_rate=drop_rate, name=name)
    # return keras.layers.Add(name=name + "output")([preact, nn])
    return keras.layers.Add(name=name + "output")([inputs, nn])


def res_mhsa(inputs, output_channel, conv_short_cut=True, strides=1, head_dimension=32, drop_rate=0, activation="gelu", name=""):
    """x ← Proj(Pool(x)) + Attention (Pool(Norm(x)))"""
    # preact = batchnorm_with_activation(inputs, activation=None, zero_gamma=False, name=name + "preact_")
    preact = layer_norm(inputs, name=name + "preact_")

    if conv_short_cut:
        shortcut = keras.layers.MaxPool2D(strides, strides=strides, padding="SAME", name=name + "shortcut_pool")(inputs) if strides > 1 else inputs
        shortcut = conv2d_no_bias(shortcut, output_channel, 1, strides=1, name=name + "shortcut_")
        # shortcut = batchnorm_with_activation(shortcut, activation=activation, zero_gamma=False, name=name + "shortcut_")
    else:
        shortcut = inputs

    nn = preact
    if strides != 1:  # Downsample
        # nn = keras.layers.ZeroPadding2D(padding=1, name=name + "pad")(nn)
        nn = keras.layers.MaxPool2D(pool_size=2, strides=strides, padding="SAME", name=name + "pool")(nn)
    num_heads = nn.shape[-1] // head_dimension
    nn = mhsa_with_multi_head_relative_position_embedding(nn, num_heads=num_heads, key_dim=head_dimension, out_shape=output_channel, name=name + "mhsa_")
    nn = drop_block(nn, drop_rate=drop_rate, name=name)
    # print(f"{name = }, {inputs.shape = }, {shortcut.shape = }, {nn.shape = }")
    return keras.layers.Add(name=name + "output")([shortcut, nn])


def CoAtNet(
    num_blocks,
    out_channels,
    stem_width=64,
    block_types=["conv", "conv", "transform", "transform"],
    strides=[2, 2, 2, 2],
    expansion=4,
    se_ratio=0.25,
    head_dimension=32,
    use_dw_strides=True,
    bn_act_first=False,  # Experiment, use activation -> BatchNorm instead of BatchNorm -> activation, also set use_bias=True for pre Conv2D layer
    input_shape=(224, 224, 3),
    num_classes=1000,
    activation="gelu",
    drop_connect_rate=0,
    classifier_activation="softmax",
    dropout=0,
    pretrained=None,
    model_name="coatnet",
    kwargs=None,
):
    img_inputs = keras.layers.Input(input_shape, name="image")
    nn = keras.layers.Rescaling(1.0 /(255) )(img_inputs)
    """ stage 0, Stem_stage """
    nn = conv2d_no_bias(nn, stem_width, 3, strides=2, use_bias=bn_act_first, padding="same", name="stem_1_")
    nn = batchnorm_with_activation(nn, activation=activation, act_first=bn_act_first, name="stem_1_")
    nn = conv2d_no_bias(nn, stem_width, 3, strides=1, use_bias=bn_act_first, padding="same", name="stem_2_")
    # nn = batchnorm_with_activation(nn, activation=activation, name="stem_2_")

    """ stage [1, 2, 3, 4] """
    total_blocks = sum(num_blocks)
    global_block_id = 0
    for stack_id, (num_block, out_channel, block_type) in enumerate(zip(num_blocks, out_channels, block_types)):
        is_conv_block = True if block_type[0].lower() == "c" else False
        stack_se_ratio = se_ratio[stack_id] if isinstance(se_ratio, (list, tuple)) else se_ratio
        stack_strides = strides[stack_id] if isinstance(strides, (list, tuple)) else strides
        for block_id in range(num_block):
            name = "stack_{}_block_{}_".format(stack_id + 1, block_id + 1)
            stride = stack_strides if block_id == 0 else 1
            conv_short_cut = True if block_id == 0 else False
            block_se_ratio = stack_se_ratio[block_id] if isinstance(stack_se_ratio, (list, tuple)) else stack_se_ratio
            block_drop_rate = drop_connect_rate * global_block_id / total_blocks
            global_block_id += 1
            if is_conv_block:
                nn = res_MBConv(
                    nn, out_channel, conv_short_cut, stride, expansion, block_se_ratio, block_drop_rate, use_dw_strides, bn_act_first, activation, name=name
                )
            else:
                nn = res_mhsa(nn, out_channel, conv_short_cut, stride, head_dimension, block_drop_rate, activation=activation, name=name)
                nn = res_ffn(nn, expansion=expansion, drop_rate=block_drop_rate, activation=activation, name=name + "ffn_")

    # nn = output_block(nn, num_classes=num_classes, drop_rate=dropout, classifier_activation=classifier_activation, act_first=bn_act_first)
    # model = keras.models.Model(inputs, nn, name=model_name)
    # add_pre_post_process(model, rescale_mode="torch")
    # reload_model_weights(model, PRETRAINED_DICT, "coatnet", pretrained, MultiHeadRelativePositionalEmbedding)
    nn = keras.layers.GlobalAveragePooling2D(name="avg_pool")(nn)
    nn = keras.layers.Dense(256, activation='relu')(nn)
    ## add the model 
    ph_inputs = keras.Input(shape=(512,), name="bettiCurve")
    
    ph_features = keras.layers.Rescaling(1.0 /(25000) )(ph_inputs)
    ph_features = keras.layers.Flatten()(ph_features)
    ph_features = keras.layers.Dense(256*2, activation='relu')(ph_features)
    for i in range(8):
      ph_features = keras.layers.Dense(256, activation='relu')(ph_features)
    
    combined_features = keras.layers.concatenate([ph_features, nn],axis=1)

    nn = combined_features
    if dropout > 0:
        nn = keras.layers.Dropout(dropout, name="head_drop")(nn)
    # outputs = keras.layers.Dense(num_classes, dtype="float32",  kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=classifier_activation, name="predictions")(nn)
    outputs = keras.layers.Dense(num_classes, dtype="float32", activation=classifier_activation, name="predictions")(nn)

    combined_model = keras.Model(
        inputs=[img_inputs,ph_inputs],
        outputs=outputs,
        name=model_name
    )
    
    # add_pre_post_process(combined_model, rescale_mode="torch")
    reload_model_weights(combined_model, PRETRAINED_DICT, "coatnet", pretrained, MultiHeadRelativePositionalEmbedding, request_resolution = 224)
    
    return combined_model


def CoAtNetT(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0, classifier_activation="softmax", **kwargs):
    num_blocks = [3, 4, 6, 3]
    out_channels = [64, 128, 256, 512]
    stem_width = 64
    return CoAtNet(**locals(), model_name="coatnett", **kwargs)


def CoAtNet0(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0, classifier_activation="softmax", pretrained="imagenet", **kwargs):
    num_blocks = [1, 1, 2,1]
    out_channels = [96, 192, 384, 768]
    stem_width = 64
    return CoAtNet(**locals(), model_name="coatnet0", **kwargs)


def CoAtNet1(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.3, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 6, 14, 2]
    out_channels = [96, 192, 384, 768]
    stem_width = 64
    return CoAtNet(**locals(), model_name="coatnet1", **kwargs)


def CoAtNet2(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.5, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 6, 14, 2]
    out_channels = [128, 256, 512, 1024]
    stem_width = 128
    return CoAtNet(**locals(), model_name="coatnet2", **kwargs)


def CoAtNet3(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.7, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 6, 14, 2]
    out_channels = [192, 384, 768, 1536]
    stem_width = 192
    return CoAtNet(**locals(), model_name="coatnet3", **kwargs)


def CoAtNet4(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.2, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 12, 28, 2]
    out_channels = [192, 384, 768, 1536]
    stem_width = 192
    return CoAtNet(**locals(), model_name="coatnet4", **kwargs)


def CoAtNet5(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.2, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 12, 28, 2]
    out_channels = [256, 512, 1280, 2048]
    stem_width = 192
    head_dimension = 64
    return CoAtNet(**locals(), model_name="coatnet5", **kwargs)


def CoAtNet6(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.2, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 4, 8, 42, 2]
    out_channels = [192, 384, 768, 1536, 2048]
    block_types = ["conv", "conv", "conv", "transfrom", "transform"]
    strides = [2, 2, 2, 1, 2]
    stem_width = 192
    head_dimension = 128
    return CoAtNet(**locals(), model_name="coatnet6", **kwargs)


def CoAtNet7(input_shape=(224, 224, 3), num_classes=1000, drop_connect_rate=0.2, classifier_activation="softmax", **kwargs):
    num_blocks = [2, 4, 8, 42, 2]
    out_channels = [256, 512, 1024, 2048, 3072]
    block_types = ["conv", "conv", "conv", "transfrom", "transform"]
    strides = [2, 2, 2, 1, 2]
    stem_width = 192
    head_dimension = 128
    return CoAtNet(**locals(), model_name="coatnet7", **kwargs)

In [ ]:
from glob import glob
data_dir = "combined_segmented_cropped_modified_resized_subfolder_80_20/train"
test_dir = "combined_segmented_cropped_modified_resized_subfolder_80_20/val"
mild = len(glob(os.path.join(data_dir+"/mild", "*.jpg")))
moderate = len(glob(os.path.join(data_dir+"/moderate", "*.jpg")))
severe = len(glob(os.path.join(data_dir+"/severe", "*.jpg")))

total  = mild+moderate+severe
weight_for_0 = (1 / mild) * (total / 3.0)
weight_for_1 = (1 / moderate) * (total / 3.0)
weight_for_2 = (1 / severe) * (total / 3.0)
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}
print(class_weight)

{0: 0.9108126721763086, 1: 0.6654088050314465, 2: 2.5047348484848486}


In [ ]:
import matplotlib.pyplot as plt

def calculate(input):
  pd = cripser.computePH(input,maxdim=1,location="birth")
  pds0_bd =np.array(pd[pd[:,0] == 0,1:3])
  pds1_bd =np.array(pd[pd[:,0] == 1,1:3])

  pds0_bd=np.clip(pds0_bd,0,255)
  pds1_bd=np.clip(pds1_bd,0,255)
  curves = representations.vector_methods.BettiCurve(256,sample_range = [0,255])
  result = np.array(curves(pds0_bd))
  result1 = np.array(curves(pds1_bd))
  return np.concatenate([result, result1],0)

image_size = (224, 224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=1337,
    image_size=image_size,
    batch_size=16,
    color_mode='rgb',
    label_mode='categorical',
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=1337,
    image_size=image_size,
    batch_size=16,
    color_mode='rgb',
    label_mode='categorical',
)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        # layers.RandomZoom(height_factor=(0.2, 0.3))
    ]
)
# train_ds = train_ds.prefetch(buffer_size=wandb.config['batch_size'])
# val_ds = val_ds.prefetch(buffer_size=wandb.config['batch_size'])
# augmented_train_ds = train_ds.prefetch(buffer_size=wandb.config['batch_size'])

augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))


train_ds_source = train_ds.unbatch()
val_ds_source = val_ds.unbatch()
augmented_val_ds = augmented_train_ds.unbatch()

ph_train_data = [];
ph_train_label = [];
image_train_data = [];
image_train_label = [];
for data,label in train_ds_source:
  ph_train_data.append(calculate(data[:,:,0]))
  ph_train_label.append(label)
  image_train_data.append(data)
  image_train_label.append(label)

ph_val_data = [];
ph_val_label = [];
image_val_data = [];
image_val_label = [];
for data,label in val_ds_source:
  ph_val_data.append(calculate(data[:,:,0]))
  ph_val_label.append(label)
  image_val_data.append(data)
  image_val_label.append(label)

tf.random.set_seed(8412)
ph_train_data=tf.random.shuffle(ph_train_data, seed=8412)
tf.random.set_seed(8412)
ph_train_label=tf.random.shuffle(ph_train_label, seed=8412)
tf.random.set_seed(8412)
image_train_data=tf.random.shuffle(image_train_data, seed=8412)
tf.random.set_seed(8412)
image_train_label=tf.random.shuffle(image_train_label, seed=8412)

ph_train_data=np.array(ph_train_data)
ph_train_label=np.array(ph_train_label)

ph_val_data=np.array(ph_val_data)
ph_val_label=np.array(ph_val_label)

image_train_data=np.array(image_train_data)
image_train_label=np.array(image_train_label)

image_val_data=np.array(image_val_data)
image_val_label=np.array(image_val_label)


Found 2645 files belonging to 3 classes.
Found 664 files belonging to 3 classes.


In [ ]:


# ph_train_ds = tf.data.Dataset.from_tensor_slices(ph_train_data)
# ph_val_ds = tf.data.Dataset.from_tensor_slices(ph_val_data)
# ph_train_label_ds = tf.data.Dataset.from_tensor_slices(ph_train_label)

# combined_train_ds = tf.data.Dataset.from_tensor_slices({"image": image_train_data,"bettiCurve":ph_train_ds},ph_train_label_ds)

# ph_train_label

In [ ]:
# assert(np.array_equal(image_train_label,ph_train_label))
train_data = {"image": image_train_data,"bettiCurve":ph_train_data}
val_data = ([image_val_data, ph_val_data], ph_val_label)

In [ ]:


# wandb.config = {
#   "learning_rate": 0.0001,
#   "epochs": 10,
#   "batch_size": 16
# }
# os.chdir("/content/drive/MyDrive/Colab Notebooks")

def make_model(input_shape, num_classes,dropout=None,config=None):

    # base_model = coatnet.CoAtNet(input_shape=input_shape, num_classes=num_classes, drop_connect_rate=config['drop_connect_rate'] , block_types=["conv", "conv", "transform"], num_blocks = [wandb.config['num_layer_1'], wandb.config['num_layer_2'], wandb.config['num_layer_3']], out_channels = [32 ,64, 128],stem_width = 64);
    base_model = CoAtNet0(input_shape,num_classes,dropout=dropout)
    return base_model;

def train():
  
  wandb.init()
  # data_dir = "data"
  input_shape=image_size + (3,)
  num_classes=3
  # model = coatnet.CoAtNet(input_shape=input_shape, num_classes=num_classes, 
  #                         drop_connect_rate=wandb.config['drop_connect_rate'] , 
  #                         block_types=["conv", "conv", "transform"], 
  #                         num_blocks = [2, 2, 2], 
  #                         out_channels = [32 ,64, 128],
  #                         stem_width = 64);
  
  model = make_model(input_shape,num_classes,dropout=wandb.config['dropout'])

  # keras.utils.plot_model(model, show_shapes=True)
  #suffle and handel randomization

  callbacks = [
      WandbCallback(),
  ]

  if wandb.config['optimizer']=='adam':
      optimizer = tf.keras.optimizers.Adam(wandb.config['learning_rate'])
  else:
      optimizer = tf.keras.optimizers.SGD(wandb.config['learning_rate'])

  model.compile(
      optimizer=optimizer,
      loss="categorical_crossentropy",
      metrics=["accuracy",
              tf.keras.metrics.SensitivityAtSpecificity(0.5,class_id=0,name="Sensitivity_Mild"),
              tf.keras.metrics.SensitivityAtSpecificity(0.5,class_id=1,name="Sensitivity_Moderate"),
              tf.keras.metrics.SensitivityAtSpecificity(0.5,class_id=2,name="Sensitivity_Severe"),
              ],
      run_eagerly=True
  )

  model.fit(
          train_data,ph_train_label, epochs=wandb.config['epochs'], batch_size=wandb.config['batch_size'], callbacks=callbacks,validation_data=val_data,
      )

In [ ]:
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
import wandb
from wandb.keras import WandbCallback
import math

sweep_config={
    'method':'bayes',
    'metric':{
        'name': 'val_accuracy',
        'goal': 'maximize'
  },
  'parameters': {
    'optimizer':{
        'values':['adam','SGD']
    },
    'dropout':{
        'min':0.0000000000000001,
        'max':1.0
    },
    'learning_rate':{          
        'min': 0.00001,
        'max': 0.001
    },
    "epochs": {
        'distribution':'int_uniform',
        'min':10,
        'max':30
    },
    "batch_size":{
      'distribution':'q_log_uniform_values',
      'q':1,
      'min':4,
      'max':8
    }
  }
}
os.chdir("/content/sample_data")

# sweep_id = wandb.sweep(sweep_config,project="final-project-results",entity="redet");
sweep_id = "3pmd91nk"
wandb.agent(sweep_id, function=train,project="final-project-results",entity="redet")

wandb: Agent Starting Run: yaw62ovw with config:
wandb: 	batch_size: 5
wandb: 	dropout: 0.33614933621752047
wandb: 	epochs: 19
wandb: 	learning_rate: 0.0007206382089706126
wandb: 	optimizer: adam
wandb: Currently logged in as: redet. Use `wandb login --relogin` to force relogin


93773824/93762208 [==============================] - 9s 0us/step
>>>> Load pretrained from: /root/.keras/models/coatnet0_224_imagenet.h5


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


>>>> Reload mismatched weights: 224 -> (244, 244)
>>>> Reload layer: stack_3_block_1_mhsa_pos_emb
>>>> Reload layer: stack_3_block_2_mhsa_pos_emb
>>>> Reload layer: stack_4_block_1_mhsa_pos_emb
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/19
529/529 [==============================] - ETA: 0s - loss: 1.0924 - accuracy: 0.4647 - Sensitivity_Mild: 0.5661 - Sensitivity_Moderate: 0.4815 - Sensitivity_Severe: 0.6392

INFO:tensorflow:Assets written to: /content/sample_data/wandb/run-20220806_210650-yaw62ovw/files/model-best/assets


...

INFO:tensorflow:Assets written to: /content/sample_data/wandb/run-20220806_213651-0ij6xjsh/files/model-best/assets
wandb: Adding directory to artifact (/content/sample_data/wandb/run-20220806_213651-0ij6xjsh/files/model-best)... Done. 0.3s


529/529 [==============================] - 96s 181ms/step - loss: 0.5643 - accuracy: 0.7573 - Sensitivity_Mild: 0.9711 - Sensitivity_Moderate: 0.9170 - Sensitivity_Severe: 0.9773 - val_loss: 0.5749 - val_accuracy: 0.7334 - val_Sensitivity_Mild: 0.9712 - val_Sensitivity_Moderate: 0.9277 - val_Sensitivity_Severe: 1.0000
Epoch 13/30
529/529 [==============================] - ETA: 0s - loss: 0.5497 - accuracy: 0.7603 - Sensitivity_Mild: 0.9773 - Sensitivity_Moderate: 0.9291 - Sensitivity_Severe: 0.9886

INFO:tensorflow:Assets written to: /content/sample_data/wandb/run-20220806_213651-0ij6xjsh/files/model-best/assets


INFO:tensorflow:Assets written to: /content/sample_data/wandb/run-20220806_213651-0ij6xjsh/files/model-best/assets
wandb: Adding directory to artifact (/content/sample_data/wandb/run-20220806_213651-0ij6xjsh/files/model-best)... Done. 0.3s


529/529 [==============================] - 96s 182ms/step - loss: 0.5497 - accuracy: 0.7603 - Sensitivity_Mild: 0.9773 - Sensitivity_Moderate: 0.9291 - Sensitivity_Severe: 0.9886 - val_loss: 0.5664 - val_accuracy: 0.7425 - val_Sensitivity_Mild: 0.9753 - val_Sensitivity_Moderate: 0.9367 - val_Sensitivity_Severe: 0.9888
Epoch 14/30
529/529 [==============================] - 88s 166ms/step - loss: 0.5361 - accuracy: 0.7705 - Sensitivity_Mild: 0.9835 - Sensitivity_Moderate: 0.9381 - Sensitivity_Severe: 0.9744 - val_loss: 0.5845 - val_accuracy: 0.7455 - val_Sensitivity_Mild: 0.9712 - val_Sensitivity_Moderate: 0.9277 - val_Sensitivity_Severe: 0.9888
Epoch 15/30
529/529 [==============================] - 88s 166ms/step - loss: 0.5280 - accuracy: 0.7788 - Sensitivity_Mild: 0.9773 - Sensitivity_Moderate: 0.9396 - Sensitivity_Severe: 0.9830 - val_loss: 0.6002 - val_accuracy: 0.7440 - val_Sensitivity_Mild: 0.9671 - val_Sensitivity_Moderate: 0.9277 - val_Sensitivity_Severe: 0.9888
Epoch 16/30
196/

In [ ]:
import gc

gc.collect()